In [1]:
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
import imageio
import os
import shutil

/home/francesco/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/francesco/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
place_name = "Copenhagen Municipality, Denmark"
graph = ox.graph_from_place(place_name , network_type='walk')

In [95]:
nodes, edges = ox.graph_to_gdfs(graph)
nodes = nodes.drop(['highway' , 'ref' , 'railway'], axis=1)
edges = edges.drop(['highway' , 'oneway' , 'reversed' , 'maxspeed'], axis=1)
edges["u"] = edges.index.map(lambda x: x[0])
edges["v"] = edges.index.map(lambda x: x[1])
central_area = nodes[
    (nodes["x"] > 12.53961)
    & (nodes["x"] < 12.57952)
    & (nodes["y"] > 55.69316)
    & (nodes["y"] < 55.70913)
]
edges = edges[
    edges["u"].isin(central_area.index) & edges["v"].isin(central_area.index)
]
subgraph = graph.subgraph(central_area.index)

In [96]:
def find_point(latitudes, longitudes):
    copy = central_area.copy()
    copy["distance"] = ox.distance.euclidean(
        copy["y"], copy["x"], latitudes, longitudes
    )
    copy = copy.sort_values("distance", ascending=True)
    return copy.index[0]

def distance(point1, point2):
    return ox.distance.euclidean(
        central_area.loc[point1]["y"],
        central_area.loc[point1]["x"],
        central_area.loc[point2]["y"],
        central_area.loc[point2]["x"],
    )

In [108]:
def next(point_before, Unique_ID, edges, arrival_point):
    possible_streets = edges[((edges.u == Unique_ID) | (edges.v == Unique_ID)) & ((edges.u != point_before) & (edges.v != point_before)) & 
                             (edges.u.isin(central_area.index) & edges.v.isin(central_area.index))]
    if len(possible_streets) == 0:
        possible_streets = edges[
            ((edges.u == Unique_ID) | (edges.v == Unique_ID)) & (
                edges.u.isin(central_area.index) & edges.v.isin(central_area.index)
            )
        ]

    x_best = central_area.loc[ arrival_point , "x"]
    y_best = central_area.loc[ arrival_point , "y"]
    xs = []
    ys = []
    for street in possible_streets.index : 
        u = edges.loc[street , "u"]
        v = edges.loc[street , "v"]
        arrive = v if u == Unique_ID else u
        if distance(arrive , arrival_point) < 0.0001:
            return arrive , street, edges.loc[street , "length"]
        elif arrive not in central_area.index:
            print("not in central area")
            xs.append(0)
            ys.append(0)
            continue
        arrive = central_area.loc[arrive]

        xs.append(arrive.x)
        ys.append(arrive.y)
    whight = 1/np.array(ox.distance.euclidean(xs ,ys  , x_best , y_best)**4)
    whight = whight / whight.sum()
    street = np.random.choice(possible_streets.index , p = whight)
    u = edges.loc[street , "u"]
    v = edges.loc[street , "v"]
    next_node = v if u == Unique_ID else u
    return next_node , street, edges.loc[street , "length"]

In [114]:
arrival_point = central_area.index[
    central_area.index.get_loc(find_point(55.69856, 12.56214))
]
def simulation(arrival_point , edges, N = 400):
    result = next(0 , central_area.index[100] , edges , arrival_point)
    line = [result[0]]
    streets = [result[1]]
    size = result[2] 
    result = next(0, line[-1] , edges , arrival_point)
    line.append(result[0])
    streets.append(result[1])
    size += result[2] 
    check = False
    for i in range(N) : 
        result = next(line[-2], line[-1] , edges , arrival_point)
        line.append(result[0])
        streets.append(result[1])
        size += result[2]
        if distance(result[0], arrival_point) < 0.0001:
            print(f"Simulation {i} arrived")
            check = True
            break
        

    if not check:
        print(f"Simulation {i} not arrived")
        size*=10000

    return size ,line ,streets

sizes, lines, streets = [], [], []
for i in range(1000):
    size, line, street = simulation(arrival_point, edges, N = 1000)
    sizes.append(size)
    lines.append(line)
    streets.append(street)

sorted_indices = np.argsort(sizes)
sizes = [sizes[i] for i in sorted_indices]
lines = [lines[i] for i in sorted_indices]
streets = [streets[i] for i in sorted_indices]
print(sizes)

arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived
arrived


In [115]:
os.makedirs("temp_images", exist_ok=True)

line = lines[0]
for i, node in enumerate(line):
    if i % 2 == 0 or i == len(line) - 1:
        path = nodes.loc[line[:i+1]]
        fig, ax = ox.plot_graph(subgraph, show=False, close=False)
        path.plot(ax=ax, color="blue", zorder=3, linewidth=2, markersize=2)
        path.iloc[[0]].plot(ax=ax, color="yellow", zorder=4, markersize=50)
        central_area.iloc[central_area.index == arrival_point].plot(ax=ax, color="green", zorder=4, markersize=50)
        path.iloc[[-1]].plot(ax=ax, color="red", zorder=4, markersize=50)

        for street in streets[0][:i+1]:
            edge = edges.loc[street]
            ax.plot(
                edge.geometry.xy[0],
                edge.geometry.xy[1],
                color="green",
                linewidth=1,
                alpha=0.7,
            )

        plt.savefig(f"temp_images/step_{i}.png")
        plt.close(fig)

images = []
for i in range(len(line)):
    if i % 2 == 0 or i == len(line) - 1:
        images.append(imageio.imread(f"temp_images/step_{i}.png"))
imageio.mimsave("evolution.gif", images, duration=0.5)
shutil.rmtree("temp_images")

/tmp/ipykernel_49570/2727095538.py:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"temp_images/step_{i}.png"))
